# 类型和类型类

在 GHCi 中可以通过 `:t` 命令查看表达式的类型，如下：

In [3]:
:t 'a'
:t "HELLO!"
:t True
:t 4 == 5
:t (True, 'a')

'a' :: Char

"HELLO!" :: [Char]

True :: Bool

4 == 5 :: Bool

(True, 'a') :: (Bool, Char)

输出结果中的 `::` 读作“**它的类型为**”。**凡是明确的类型，其首字母必为大写。**

编写函数时，给它一个明确的类型声明是个好习惯，如下：

In [7]:
addThree :: Int -> Int -> Int -> Int
addThree x y z = x + y + z

类型之间由 `->` 分隔，返回值始终是最后一项。

## 常见类型

以下是 Haskell 中的一些常见类型：
- `Int`：有界整数
- `Integer`：无界整数
- `Float`：单精度浮点数
- `Double`：双精度浮点数
- `Bool`：布尔值
- `Char`：字符

## 类型变量

以 `head` 函数和 `fst` 函数的类型为例，我们会发现：

In [4]:
:t head
:t fst

head :: forall a. [a] -> a

fst :: forall a b. (a, b) -> a

其中 `a` 或 `b` 表示类型变量，意味着 `a` 或 `b` 可以是任意的类型，类型变量的命名通常使用单个字符，如 `a`，`b`，`c`，`d`...

## 类型类

以 `==` 函数的类型为例，我们会发现：

In [5]:
:t (==)
-- 使用:t命令查看函数的类型时，必须以前缀函数的形式调用它，因此使用(==)

(==) :: forall a. Eq a => a -> a -> Bool

其中 `=>` 符号左边的部分叫做类型约束，这段类型声明可以解读为：“相等函数取两个相同类型的值作为参数并返回一个布尔值，而这两个参数的类型同在 `Eq` 类之中（即类型约束）”

以下是 Haskell 中的一些常见的类型类：

### `Eq`

`Eq` 类型类包含可判断相等性的类型。提供实现的函数是 `==` 和 `/=`。除函数以外的所有类型都属于 `Eq`，所以它们都可以判断相等性。

### `Ord`

`Ord` 类型类包含可比较大小的类型。包含了 `<`，`>`，`<=`，`>=` 之类用于比较大小的函数。除函数外，目前所谈到的所有类型都属于 `Ord` 类。类型若要成为 `Ord` 的成员，必先加入 `Eq` 家族。

In [14]:
:t (>)

(>) :: forall a. Ord a => a -> a -> Bool

`compare` 函数取两个 `Ord` 类中的相同类型的值作参数，返回比较的结果。这个结果是如下三种类型之一：`GT`,`LT`,`EQ`。

In [23]:
5 `compare` 3

GT

### `Show`

`Show` 类型类包含可用字符串表示的类型。操作 `Show` 类型类，最常用的函数是 `show`。它可以取任一 `Show` 的成员类型并将其转为字符串。目前为止，除函数以外的所有类型都是 `Show` 的成员。

In [22]:
:t show
show 3
show 5.334
show True

show :: forall a. Show a => a -> String

"3"

"5.334"

"True"

### `Read`

与 `Show` 相反的类型类。`read` 函数可以将一个字符串转为 `Read` 的某成员类型。

In [19]:
read "5" - 2

3

注意：若直接执行 `read 5` 会报错，这是因为 `5` 既可以是 `Int` 类型也可以是 `Float` 类型，此时必须在表达式后跟 `::` 的类型注释，以明确其类型。如下：

In [21]:
:t read
read "5" :: Int
read "5" :: Float

read :: forall a. Read a => String -> a

5

5.0

### `Enum`

`Enum` 类型类包含可枚举的类型。该类型类包含的类型有：`()`,`Bool`,`Char`,`Ordering`,`Int`,`Integer`,`Float` 和 `Double`。

### `Bounded`

`Bounded` 类型类其成员都有上限和下限。可以使用 `minBound` 或 `maxBound` 函数来查看某个类型的下限或上限。

In [15]:
:t minBound
(minBound :: Int)
(maxBound :: Char)
(maxBound :: Bool)
(minBound :: Bool)

minBound :: forall a. Bounded a => a

-9223372036854775808

'\1114111'

True

False

### `Num`

`Num` 类型类包含具有数字特征的类型。其包含所有数字：实数和整数。类型只有亲近 `Show` 和 `Eq`，才可以加入 `Num`。

In [17]:
:t 20
:t (*)

20 :: forall p. Num p => p

(*) :: forall a. Num a => a -> a -> a

### `Integral`

`Integral` 类型类仅包含整数，其成员类型有 `Int` 和 `Integer`。

### `Floating`

`Floating` 类型类仅包含浮点类型：`Float` 和 `Double`。

如果执行下面的语句会发生什么呢？

In [6]:
length [1,2,3,4] + 3.2

: 

之所以会报错，是因为 `length` 函数返回的值是 `Int` 类型，而 `3.2` 是 `Float` 类型：

In [8]:
:t length
:t 3.2

length :: forall (t :: * -> *) a. Foldable t => t a -> Int

3.2 :: forall p. Fractional p => p

要解决这种问题，可以使用 `fromIntegral` 函数，它取一个整数做参数并返回一个更加通用的数字，这在同时处理整数和浮点时会尤为有用。

In [10]:
:t fromIntegral
fromIntegral (length [1,2,3,4]) + 3.2

fromIntegral :: forall a b. (Integral a, Num b) => a -> b

7.2